In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])

In [2]:
import ee
import geemap
import geemap.foliumap as fgmap
import datapane as dp
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import sys
from ipywidgets import Button, DatePicker , Layout
from ipyleaflet import WidgetControl, ImageOverlay
import ipywidgets as widgets
import uuid
#import folium
import json
from geemap import geojson_to_ee
import geojson
from geojson import GeometryCollection, Point, LineString
#from ipyleaflet import ImageOverlay

In [3]:
conn = ps.connect(dbname="postgres",
                  user="postgres",
                  password="vssql",
                  host="localhost",
                  port="5432")

In [4]:
import warnings

warnings.filterwarnings('ignore')

cur = conn.cursor()

In [5]:
try:
    service_account = ' totvs-9382f3354ed5e4054e9feb16b@ee-daniel97appi.iam.gserviceaccount.com'
    credentials = ee.ServiceAccountCredentials(service_account, 'ee-token.json')
    ee.Initialize(credentials)
    print('Google Earth Engine has initialized succesfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialized!')
except:
    print('Unexpected error:', sys.exc_info()[0])
    raise

Google Earth Engine has initialized succesfully!


In [6]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap('HYBRID')  # Add Google Satellite
Map.setCenter(-7.03125, 31.0529339857, 2)
mc = Map.measure_control
toolbarButton = Map.toolbar_button
toolbar = Map.toolbar
buttonToolbar = Map.toolbar_reset
Map.remove_control(mc)

In [7]:
# Create the button widget
load_button = Button(description="Filtrar Data")

# Create the date picker widget
start_date_picker = DatePicker(description='Data Inicial:')
end_date_picker = DatePicker(description='Data Final:')

collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)).filter(ee.Filter.lt('CLOUD_COVER', 10))
img_median = collection.median()

#img = img_median.first()
img = None

# Create a variable to store the layer
layer = None

# Define a function to handle the button click event
def on_button_click(b, collection):
    if not start_date_picker.value or not end_date_picker.value:
        from datetime import datetime, timedelta
        yesterday = datetime.now() - timedelta(days=1)
        start_date = yesterday.strftime('%Y-%m-%d')
        end_date = yesterday.strftime('%Y-%m-%d')
    else:
        start_date = start_date_picker.value.strftime('%Y-%m-%d')
        end_date = end_date_picker.value.strftime('%Y-%m-%d')
    start_date = ee.Date(start_date)
    end_date = ee.Date(end_date)   
    
    #Filter the collection to the date range    
    collection = collection.filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10));
    print(start_date, end_date)
    img = ee.Image(collection.first())
    calcIndices(img)   

# Display the button and date picker widgets
display(load_button)
display(start_date_picker)
display(end_date_picker)

# Create the WidgetControl
buttonDate = WidgetControl(widget=start_date_picker, position='topright')
buttonDate2 = WidgetControl(widget=end_date_picker, position='topright')
buttonLoad = WidgetControl(widget=load_button, position='topright')

# Add the WidgetControl to the map
Map.add_control(buttonDate)
Map.add_control(buttonDate2)
Map.add_control(buttonLoad)

# Function to calculate NDVI and GARI
def calcIndices(img):
    nir = img.select('B5')
    red = img.select('B4')
    green = img.select('B3')
    blue = img.select('B2')
    
    # Calculate NDVI
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
    
    gari = green.subtract(red).divide(green.add(red.subtract(blue))).multiply(green.subtract(nir)).divide(green.add(nir)).rename('GARI');
    
    evi = ((nir.subtract(red)).divide(nir.add(6)).multiply(red.subtract(7.5).multiply(blue.add(1))).multiply(2.5)).rename('EVI');

    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI');

    gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI');

    ndre = nir.subtract(red).divide(nir.add(red)).rename('NDRE');

    osavi = nir.subtract(red).divide(nir.add(red).add(0.16)).rename('OSAVI');

    gsavi = (nir.subtract(green).divide(nir.add(green).add(0.5)).divide(1.5)).rename('GSAVI');

    navi = (red.divide(nir)).divide(1).rename('NAVI');

    rvi = nir.divide(red).rename('RVI');

    savi = (nir.subtract(red).divide(nir.add(red).add(0.5))).multiply(1.5).rename('SAVI');

    return img.addBands(ndvi).addBands(gari).addBands(evi).addBands(ndwi).addBands(gndvi).addBands(ndre).addBands(osavi).addBands(gsavi).addBands(navi).addBands(rvi).addBands(savi)

# Map the calcIndices function over the collection
indices_collection = collection.map(calcIndices)
#load_button.on_click(lambda b: on_button_click(b, collection))

# Attach the function to the button's click event
load_button.on_click(lambda b: on_button_click(b, collection))

# Get the median NDVI image
median_ndvi = indices_collection.select('NDVI').median()
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

# Get the median GARI image
median_gari = indices_collection.select('GARI').median()
median_evi = indices_collection.select('EVI').median()
median_ndwi = indices_collection.select('NDWI').median()
median_gndvi = indices_collection.select('GNDVI').median()
median_ndre = indices_collection.select('NDRE').median()
median_osavi = indices_collection.select('OSAVI').median()
median_gsavi = indices_collection.select('GSAVI').median()
median_navi = indices_collection.select('NAVI').median()
median_rvi = indices_collection.select('RVI').median()
median_savi = indices_collection.select('SAVI').median()

gariParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

indexParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

Button(description='Filtrar Data', style=ButtonStyle())

DatePicker(value=None, description='Data Inicial:', step=1)

DatePicker(value=None, description='Data Final:', step=1)

In [8]:
index_options = ['NDVI', 'GARI', 'EVI', 'NDWI', 'GNDVI', 'NDRE', 'OSAVI', 'GSAVI', 
                 'NAVI', 'RVI', 'SAVI']

# Create a dropdown widget for the index options
index_dropdown = widgets.Dropdown(options=index_options)
display(index_dropdown)

# Map the index names to the index images and parameters
index_dict = {
    'NDVI': (median_ndvi, ndviParams),
    'GARI': (median_gari, gariParams),
    'EVI': (median_evi, indexParams),
    'NDWI': (median_ndwi, indexParams),
    'GNDVI': (median_gndvi, indexParams),
    'NDRE': (median_ndre, indexParams),
    'OSAVI': (median_osavi, indexParams),
    'GSAVI': (median_gsavi, indexParams),
    'NAVI': (median_navi, indexParams),
    'RVI': (median_rvi, indexParams),
    'SAVI': (median_savi, indexParams),          
}

# Create the WidgetControl
control = WidgetControl(widget=index_dropdown, position='topright')

Map.add_control(control)

Dropdown(options=('NDVI', 'GARI', 'EVI', 'NDWI', 'GNDVI', 'NDRE', 'OSAVI', 'GSAVI', 'NAVI', 'RVI', 'SAVI'), va…

In [9]:
# Get basemap layers
base_layers = Map.layers

feat_list = []


# Create the draw control
dc = Map.draw_control
dc.rectangle = {"shapeOptions": {"fillColor": "transparent", "color": "black"}}
dc.polygon = {"shapeOptions": {"fillColor": "transparent", "color": "black"},"options": {"title": "Custom Title for Rectangle"}}
dc.circle = {"shapeOptions": {"fillColor": "transparent", "color": "black"}}
dc.edit = True
dc.remove = False #trash button 

# Create a dictionary to store the features and images for each index type
images = {}

data_to_insert = []

removeShapes = []

clip_imageGlobal = None
index_paramsGlobal = None
selected_indexGlobal = None

shape_dict = {}

# Handle draw events
def handle_draw(self, action, geo_json):
    
    global clip_image
    global index_params
    global selected_index
    
    selected_index = index_dropdown.value

    # Get the index image and parameters
    index_image, index_params = index_dict[selected_index]

    geom = geojson_to_ee(geo_json, False)

    feature = ee.Feature(geom)
    feature = feature.set("index", str(selected_index))
    feature = feature.set("selected", False)
    feature = feature.set("id", str(uuid.uuid4()))

    geojson_dict = geom.getInfo()
    geojsonStr = geojson.dumps(geojson_dict)
    data_to_insert.append((geojsonStr, str(selected_index)))
    
    feature = ee.Feature(geom)
        
    if selected_index in images:
        feat_list, img_list = images[selected_index]
        feat_list.append(feature)
        img_list.append((index_image, index_params))
    else:
        images[selected_index] = ([feature], [(index_image, index_params)])

    # Add only the new images to the map
    feat_list, img_list = images[selected_index]
    collection = ee.FeatureCollection(feat_list)
    clip_image = index_image.clipToCollection(collection)

    
    removeShapes.append((clip_image, index_params, selected_index))

    Map.add_ee_layer(clip_image, index_params, selected_index)
    
dc.on_draw(handle_draw)

In [10]:
def savePlot(s):
    cur.executemany("INSERT INTO geomapping (descricao, indice) VALUES (%s,%s)", data_to_insert)
    conn.commit()
    
load_data_button = Button(description="Salvar indices")
load_data_button.on_click(savePlot)

buttonSave = WidgetControl(widget=load_data_button, position='topright')

Map.add_control(buttonSave)

def load_data_from_db(b):
   
# Fetch the data from the database
    cur.execute("SELECT descricao, indice FROM geomapping")
    rows = cur.fetchall()

    # Iterate over the rows and display the data on the map
    for row in rows:
        geojson_str, index_selected = row
        # Convert the geojson data from the database to an ee.Geometry object
        geojson_data = json.loads(geojson_str)
        geom = ee.Geometry.Polygon(geojson_data['coordinates'])
        index_image, index_params = index_dict[index_selected]
        clipped_image = index_image.clip(geom)
        Map.add_ee_layer(clipped_image, index_params, index_selected)
        
            
load_data_button = Button(description="Carregar indices")
load_data_button.on_click(load_data_from_db)

buttonLoad = WidgetControl(widget=load_data_button, position='topright')
#['topright', 'topleft', 'bottomright', 'bottomleft']

Map.add_control(buttonLoad)

def delLayer(q):
    Map.remove_drawn_features()
    Map.remove_labels()
    Map.clear_layers()
    Map.add_basemap('HYBRID')  # Add Google Satellite
    Map.setCenter(-48.86154371799718, -21.53706897593839, 6)
    
delButton = Button(description="", icon="fa-solid fa-trash", layout=Layout(width='30px'))
delButton.on_click(delLayer)

layerDel = WidgetControl(widget=delButton, position='topleft')
Map.add_control(layerDel)

Map

Map(center=[31.0529339857, -7.03125], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

#### 

SyntaxError: invalid syntax (3915903684.py, line 1)